### FE545 - Homework #4

**Author**: Sid Bhatia

**Date**: April 6th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Steve Yang

#### Problem 1

Pricing a derivative security entails calculating the *expected discounted value* of its *payoff*. This reduces, in principle, to a problem of numerical integration; but in practice this calculation is often difficult for high-dimensional pricing problems. 

**Broadie** and **Glasserman** (1997)  proposed the method of the simulated tree to price American options, which can derive the upper and lower bounds for American options. This combination makes it possible to measure and control errors as the computational effort increases. 

The main drawback of the random tree method is that its computational requirements grow exponentially in the number of exercise dates $m$, so the method is applicable only when $m$ is small. Nevertheless, for problems with small $m$, it is very effective, and it also serves to illustrate a theme of managing sources of high and low bias.

Let $\tilde{h_i}$ denote the payoff function for exercise at $t_i$ which we now allow to depend on $i$. Let $\tilde{V}_i(x)$ denote the value of the option at $t_i$ given $X_i = x_i$ (the option has not been exercised). We are ultimately interested in $\tilde{V}_0(X_0).$ This value is determined recursively as follows:

$$\tilde{V}_m(x) = \tilde{h}_m(x) \tag{1}$$

$$\tilde{V}_{i-1}(x) = \max\{\tilde{h}_{i-1}(x), \; \mathbb{E}[D_{i-1, i}(X_i) \cdot \tilde{V}_i(X_i) \mid X_{i-1} = x]\} \tag{2}$$